### Env

In [6]:
# Env
import nibabel as nib
from nilearn import datasets, input_data, plotting, connectome, image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr, zscore
import pandas as pd
import os
import json
import glob
import time
import gc
import time
from joblib import Parallel, delayed

# Functions
def load_image(sub, movie):
    path = os.path.join(f'/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_datasets/ds002837/derivatives/sub-{sub}/func', 
                        f'sub-{sub}_task-{movie}_bold_blur_censor_ica.nii.gz')
    return nib.load(path, mmap=True)


def save_correlation_data(networks_data, sub, movie): 
    # format dataframe
    df = pd.DataFrame(networks_data, columns=['network', 'start', 'end', 
                                              'net_mean', 'net_median','net_std','net_mean_abs','net_median_abs','net_std_abs',
                                              'fc_mean_abs','fc_median_abs','fc_std_abs'])
    df['sub'] = sub
    df['movie'] = movie
    
    # file name
    base_filename = f'/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_derived/networks_data_v2/sub-{sub}_task-{movie}_networks'
    version = 1
    filename = f"{base_filename}_v{version}.parquet"
    
    # check if there is a previous version
    while os.path.exists(filename):
        version += 1
        filename = f"{base_filename}_v{version}.parquet"
        
    df.to_parquet(filename)


def brain_networks_extraction(df_networks, network_name, fmri_img, radius = 6):
    # Coordinates
    peak_coords = df_networks[df_networks.name == network_name][['x', 'y', 'z']].values
    # Create 6mm spheres around these coordinates
    spheres_masker = input_data.NiftiSpheresMasker(seeds=peak_coords, radius=radius, standardize=True)
    # Extract time series data for each sphere
    time_series = spheres_masker.fit_transform(fmri_img) 
    return time_series


def network_statistic(time_series):
    net_stats = [np.mean(time_series), 
                 np.median(time_series), 
                 np.std(time_series),
                 np.mean(abs(time_series)), 
                 np.median(abs(time_series)), 
                 np.std(abs(time_series))]
    return net_stats
    

def functional_conectivity_statistic(time_series):
    # Compute the functional connectome using ConnectivityMeasure
    correlation_measure = connectome.ConnectivityMeasure(kind='correlation')
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]
    fc_stats = [np.mean(abs(correlation_matrix)), 
                np.median(abs(correlation_matrix)), 
                np.std(abs(correlation_matrix))]
    return fc_stats


def process_time_intervals(img_original, network_list, start, end, step, radius=2):
    """ Extract data of interval using brain_networks_extraction, network_statistic and functional_conectivity_statistic """
    
    networks_data = []
    for t in range(start, end, step):
        img = img_original.dataobj[:,:,:,t:t+step]
        img = nib.Nifti1Image(img, img_original.affine, img_original.header)
        # here gonna come the function that change the image for mni space
        
        for network_name in network_list:
            time_series = brain_networks_extraction(df_networks, network_name=network_name, fmri_img=img, radius=radius)
            net_stats = network_statistic(time_series)
            fc_stats = functional_conectivity_statistic(time_series)
            networks_data.append([network_name, t, t+step] + net_stats + fc_stats) 
            #print(f"{network_name} | clip start: {t} | clip end: {t+step}")
    
    return networks_data


def process_participant(sub, movie, network_list, end, duration, step):
    print(f"Initializing processing participant {sub} | {movie}")
    
    # Load: 153 ms
    img_original = load_image(sub, movie)
    
    # Process: 2s per interval per network
    start = end - duration
    networks_data = process_time_intervals(img_original, network_list, start=start, end=end, step=step)
    
    # Save: 600ms
    save_correlation_data(networks_data, sub, movie)
    print(f"File saved participant {sub} | {movie}")

In [13]:
# Data
df_networks = pd.read_csv("/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_derived/mni_space_of_networks.csv")
participants = pd.read_csv("/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_datasets/ds002837/participants.tsv", sep='\t')
participants['sub'] = range(1,87)
participants = participants[participants['sub'] != 49] # ta corrompido
participants['end_movie'] = [load_image(sub=row['sub'], movie=row['task']).shape[3] for index, row in participants.iterrows()]


# Parameters
participants_test = participants.iloc[0:2]
network_list = ['Autobiographical memory',
                 'Theory of mind',
                 'Cognitive attention control',
                 'Emotional scene and face processing', 
                 'Extended socio-affective default network',
                 'Reward',
                 'Empathy']
duration = 20 #30 * 60
step = 5
start_counting = 30*60 # end of movie and end of interval are two different thing

# Set the number of jobs (parallel workers)
n_jobs = 2  # Adjust this number based on your system's capacity
os.environ["OMP_NUM_THREADS"] = "1"  # Ensure thread limiting if needed
results = Parallel(n_jobs=n_jobs)(
    delayed(process_participant)(sub, movie, network_list, (end_movie - start_counting), duration, step) 
    for sub, movie, end_movie in participants_test[['sub', 'task','end_movie']].values
)

Initializing processing participant 2 | 500daysofsummer
Initializing processing participant 1 | 500daysofsummer
File saved participant 1 | 500daysofsummer
File saved participant 2 | 500daysofsummer


In [12]:
participants[participants.task=='500daysofsummer']

,participant_id,age,sex,task,sub,end_movie
0,sub-1,23,M,500daysofsummer,1,5470
1,sub-2,25,F,500daysofsummer,2,5470
2,sub-3,23,M,500daysofsummer,3,5470
3,sub-4,23,M,500daysofsummer,4,5470
4,sub-5,22,F,500daysofsummer,5,5470
5,sub-6,25,M,500daysofsummer,6,5470
6,sub-7,23,F,500daysofsummer,7,5470
7,sub-8,23,F,500daysofsummer,8,5470
8,sub-9,28,F,500daysofsummer,9,5470
9,sub-10,31,M,500daysofsummer,10,5470


In [11]:
pd.read_parquet('/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_derived/networks_data_v2/sub-1_task-500daysofsummer_networks_v1.parquet')

,network,start,end,net_mean,net_median,net_std,net_mean_abs,net_median_abs,net_std_abs,fc_mean_abs,fc_median_abs,fc_std_abs,sub,movie
0,Autobiographical memory,3650,3655,-1.969545e-08,0.048349,1.0,0.849293,0.769349,0.527922,0.241510,0.222288,0.200482,1,500daysofsummer
1,Theory of mind,3650,3655,-1.430511e-08,-0.030876,1.0,0.840742,0.861385,0.541435,0.181263,0.119844,0.230365,1,500daysofsummer
2,Cognitive attention control,3650,3655,1.631285e-08,0.219747,1.0,0.883131,0.847528,0.469127,0.254617,0.233904,0.214005,1,500daysofsummer
3,Emotional scene and face processing,3650,3655,-7.450581e-10,-0.036314,1.0,0.838040,0.809158,0.545608,0.251080,0.229340,0.199804,1,500daysofsummer
4,Extended socio-affective default network,3650,3655,-1.390775e-08,-0.052597,1.0,0.844863,0.771918,0.534982,0.195347,0.119507,0.252900,1,500daysofsummer
5,Reward,3650,3655,-5.722046e-09,-0.038789,1.0,0.845395,0.786060,0.534141,0.254650,0.240495,0.199299,1,500daysofsummer
6,Empathy,3650,3655,-1.083721e-08,0.171924,1.0,0.836153,0.746948,0.548496,0.170389,0.130393,0.199777,1,500daysofsummer
7,Autobiographical memory,3655,3660,8.292820e-09,0.005205,1.0,0.855579,0.874364,0.517672,0.256916,0.223804,0.201218,1,500daysofsummer
8,Theory of mind,3655,3660,-1.589457e-09,0.210160,1.0,0.862768,0.794648,0.505601,0.264742,0.238738,0.229435,1,500daysofsummer
9,Cognitive attention control,3655,3660,-1.631285e-08,-0.147168,1.0,0.840566,0.826242,0.541709,0.247503,0.203090,0.213962,1,500daysofsummer
